In [5]:
# import cPickle as pickle
import ast
import pickle
import re

import numpy as np
import pandas as pd

import json

In [6]:
import findspark

findspark.init()

In [7]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession


spark = SparkSession\
        .builder\
        .appName("TrainingOnCombinedSummary")\
        .config('spark.sql.warehouse.dir','file:///home/sudipto21048867/Pyspark/spark-warehouse')\
        .config("spark.sql.caseSensitive","true")\
        .getOrCreate()

In [8]:
#data = json.load(open('C:/Users/sudsaha1/Documents/My Received Files/TransformedSummary.json'))

#TransactionDf_pd = [item for items in data.values() for item in items]
#TransactionDf_pd=pd.DataFrame(TransactionDf_pd) 


#TransactionDf_pd.head()
TransactionDf_pd = pd.read_csv('C:/Users/sudsaha1/Documents/My Received Files/CombinedSummary.csv')

In [9]:
TransactionDf=spark.createDataFrame(TransactionDf_pd)
#pd.DataFrame(TransactionDf.take(5),columns=TransactionDf.columns).transpose()

In [ ]:
#TransactionDf_pd.dtypes

In [ ]:
#sc = spark.sparkContext
#Transaction_rdd = sc.textFile("D:/Llyods/BASE_Data/CombinedSummary.csv")
#Transaction_rdd.take(5)
#TransactionDf.dtypes

In [10]:
from pyspark.sql.types import FloatType
from pyspark.sql.types import StringType
from pyspark.sql.functions import UserDefinedFunction as udf
from pyspark.sql.functions import expr
from pyspark.sql import SQLContext, Row

In [ ]:
TransactionDf_pd['AmountInGBP'] = TransactionDf_pd['Amount'].map(lambda x: ast.literal_eval(x)['Amount']).astype(np.float)
TransactionDf_pd['Type'] = TransactionDf_pd['BankTransactionCode'].map(lambda x: ast.literal_eval(x[:-1] + "\"\"" + '}')['Code'])

In [ ]:
TransactionDf_pd.head()

In [11]:
from pyspark.sql.types import FloatType
from pyspark.sql.types import StringType
from pyspark.sql.functions import UserDefinedFunction as udf

In [12]:

AmountInGBP = udf(lambda x: (x.split(",")[0].split(":")[1]).replace('"', '').strip() , StringType())

Type = udf(lambda x: (x.split(",")[0].split(":")[1]).replace('"', '').strip() , StringType())

def string_to_float(x):
    return float(x)

udfstring_to_float = udf(string_to_float, FloatType())
#paid_out_cleaned = udf(lambda y: y.replace(',', '') if y!='-' else '0', StringType())

In [13]:

TransactionDf = TransactionDf.withColumn('AmountInGBP',udfstring_to_float(AmountInGBP(TransactionDf['Amount']))).\
                              withColumn('Type',Type(TransactionDf['BankTransactionCode']))

In [14]:
TransactionDf.printSchema()

root
 |-- Unnamed: 0: long (nullable = true)
 |-- AccountId: long (nullable = true)
 |-- Amount: string (nullable = true)
 |-- BankTransactionCode: string (nullable = true)
 |-- BookingDateTime: string (nullable = true)
 |-- CreditDebitIndicator: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- TransactionId: long (nullable = true)
 |-- TransactionInformation: string (nullable = true)
 |-- ValueDateTime: string (nullable = true)
 |-- category: string (nullable = true)
 |-- Detailed Category: string (nullable = true)
 |-- AmountInGBP: float (nullable = true)
 |-- Type: string (nullable = true)



In [ ]:
pd.DataFrame(TransactionDf.take(5),columns=TransactionDf.columns).transpose()

In [15]:
import numpy as np



def BinningData(TransactionDf,bins):
    max_val =TransactionDf.select(TransactionDf.AmountInGBP).rdd.max()[0] 
    min_val =TransactionDf.select(TransactionDf.AmountInGBP).rdd.min()[0] -1  ##??
    custom_bucket_array = np.linspace(min_val, max_val, 4)
    cut_points = list(custom_bucket_array)
    return cut_points

In [16]:
from pyspark.ml.feature import Bucketizer
bucketizer = Bucketizer(splits=BinningData(TransactionDf,4),inputCol="AmountInGBP", outputCol="AmountInGBP_bin")
TransactionDf = bucketizer.transform(TransactionDf)

#TransactionDf.show()

In [ ]:
#pd.DataFrame(TransactionDf.take(5),columns=TransactionDf.columns).transpose()

In [17]:
t = {0:"low", 1: "medium", 2:"high"}
map_bins = udf(lambda x: t[x], StringType())
TransactionDf = TransactionDf.withColumn("AmountInGBP_bin", map_bins("AmountInGBP_bin"))

In [ ]:
pd.DataFrame(TransactionDf.take(5),columns=TransactionDf.columns).transpose()

In [18]:
# Get Merchant info from Description
def getMerchfromDesc(tranType, tranDesc):
    if tranType == 'POS':
        parsed = tranDesc.split(', ')
        lenPar = len(parsed)
        if parsed[lenPar - 1] == 'REFUND':
            merch = parsed[1:-2]
            return ('').join(merch) + 'REFUND'
        else:
            merch = parsed[1:-1]
            return ('').join(merch)
    else:
        return tranDesc
    
import re    
    
# Clean Merchant info
def cleanMerchant(txt):
    txt = re.sub(r'[^a-zA-Z\b]', ' ', str(txt).upper())
    return " ".join(txt.split())    

In [19]:
parse_merchant = udf(getMerchfromDesc , StringType())
parse_cleanMerchant = udf(cleanMerchant , StringType())
#Type = udf(lambda x: (x.split(",")[0].split(":")[1]).replace('"', '').strip() , StringType())



TransactionDf = TransactionDf.withColumn('Merchant',parse_merchant(TransactionDf['Type'],TransactionDf['TransactionInformation']))
#                              .withColumn('cleanedMerchant',parse_cleanMerchant(TransactionDf['Merchant']))
TransactionDf = TransactionDf.withColumn('cleanedMerchant',parse_cleanMerchant(TransactionDf['Merchant']))

In [ ]:
TransactionDf_pd.columns

In [ ]:
pd.DataFrame(TransactionDf.take(2),columns= TransactionDf.columns).transpose()

In [ ]:
#TransactionDf['ConcatCategory'] = TransactionDf['category'] + ', ' + TransactionDf['Detailed Category'] ???



In [20]:
def Predictorcolumn(CreditDebitIndicator,cleanedMerchant,Type,AmountInGBP_bin):
    return CreditDebitIndicator+ ' ' +cleanedMerchant+ ' ' +Type+ ' ' +AmountInGBP_bin

def ConcatCategory(category,Detailed_Category):
    return category+ ','+Detailed_Category

ConcatCategory_udf = udf(ConcatCategory,StringType())

Predictorcolumn_udf = udf(Predictorcolumn,StringType())    
    

In [21]:
TransactionDf = TransactionDf.withColumn('Predictorcolumn',Predictorcolumn_udf(TransactionDf['CreditDebitIndicator'],TransactionDf['cleanedMerchant'],TransactionDf['Type'],TransactionDf['AmountInGBP_bin']))
TransactionDf = TransactionDf.withColumn('ConcatCategory',ConcatCategory_udf(TransactionDf['category'],TransactionDf['Detailed Category']))

In [ ]:
pd.DataFrame(TransactionDf.take(2),columns= TransactionDf.columns).transpose()

In [22]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.feature import NGram
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier 
from pyspark.ml.feature import StringIndexer, VectorIndexer, VectorAssembler, SQLTransformer
from pyspark.sql.functions import *

In [23]:
train = TransactionDf.select(col("Predictorcolumn"),col("ConcatCategory"))

In [24]:
#train.head()

In [25]:
train = Tokenizer(inputCol="Predictorcolumn", outputCol="words").transform(train)
#train.head()

In [26]:
train = StopWordsRemover(inputCol="words", outputCol="words_no_stop").transform(train)
#train.head()

In [27]:
train = NGram(n=2, inputCol="words_no_stop", outputCol="ngrams_words").transform(train)
#train.head()

In [28]:
train = HashingTF(inputCol="ngrams_words", outputCol="rawFeatures").transform(train)
#train.head()


In [29]:
train = IDF(inputCol="rawFeatures", outputCol="features").fit(train).transform(train)
#train.head()

In [30]:
train = StringIndexer(inputCol="ConcatCategory", outputCol="indexedConcatCategory").fit(train).transform(train)
#train.head()

In [31]:

train = VectorAssembler(inputCols=["features"],outputCol="vector_feature").transform(train)
#train.head()

In [32]:

rf = RandomForestClassifier(labelCol="indexedConcatCategory", featuresCol="vector_feature").fit(train).transform(train)

rf.select(col("prediction"),col("probability")).show(5)

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 57822)
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "C:\ProgramData\Anaconda3\lib\socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "C:\ProgramData\Anaconda3\lib\socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "C:\ProgramData\Anaconda3\lib\socketserver.py", line 696, in __init__
    self.handle()
  File "D:\spark\spark-2.2.0-bin-hadoop2.7\python\pyspark\accumulators.py", line 235, in handle
    num_updates = read_int(self.rfile)
  File "D:\spark\spark-2.2.0-bin-hadoop2.7\python\pyspark\serializers.py", line 575, in read_int
    length = stream.read(4)
  File "C:\ProgramData\Anaconda3\lib\socket.py", line 586, in readinto
    re

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:57693)
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2910, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-32-3933876fc135>", line 2, in <module>
    rf = RandomForestClassifier(labelCol="indexedConcatCategory", featuresCol="vector_feature").fit(train).transform(train)
  File "D:\spark\spark-2.2.0-bin-hadoop2.7\python\pyspark\ml\base.py", line 64, in fit
    return self._fit(dataset)
  File "D:\spark\spark-2.2.0-bin-hadoop2.7\python\pyspark\ml\wrapper.py", line 265, in _fit
    java_model = self._fit_java(dataset)
  File "D:\spark\spark-2.2.0-bin-hadoop2.7\python\pyspark\ml\wrapper.py", line 262, in _fit_java
    return self._java_obj.fit(dataset._jdf)
  File "D:\spark\spark-2.2.0-bin-hadoop2.7\python\lib\py4j-0.10.4-src.zip\py4j\java_gateway.py", line 1133, in __call__
  

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:57693)
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2910, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-32-3933876fc135>", line 2, in <module>
    rf = RandomForestClassifier(labelCol="indexedConcatCategory", featuresCol="vector_feature").fit(train).transform(train)
  File "D:\spark\spark-2.2.0-bin-hadoop2.7\python\pyspark\ml\base.py", line 64, in fit
    return self._fit(dataset)
  File "D:\spark\spark-2.2.0-bin-hadoop2.7\python\pyspark\ml\wrapper.py", line 265, in _fit
    java_model = self._fit_java(dataset)
  File "D:\spark\spark-2.2.0-bin-hadoop2.7\python\pyspark\ml\wrapper.py", line 262, in _fit_java
    return self._java_obj.fit(dataset._jdf)
  File "D:\spark\spark-2.2.0-bin-hadoop2.7\python\lib\py4j-0.10.4-src.zip\py4j\java_gateway.py", line 1133, in __call__
  

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:57693)
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2910, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-32-3933876fc135>", line 2, in <module>
    rf = RandomForestClassifier(labelCol="indexedConcatCategory", featuresCol="vector_feature").fit(train).transform(train)
  File "D:\spark\spark-2.2.0-bin-hadoop2.7\python\pyspark\ml\base.py", line 64, in fit
    return self._fit(dataset)
  File "D:\spark\spark-2.2.0-bin-hadoop2.7\python\pyspark\ml\wrapper.py", line 265, in _fit
    java_model = self._fit_java(dataset)
  File "D:\spark\spark-2.2.0-bin-hadoop2.7\python\pyspark\ml\wrapper.py", line 262, in _fit_java
    return self._java_obj.fit(dataset._jdf)
  File "D:\spark\spark-2.2.0-bin-hadoop2.7\python\lib\py4j-0.10.4-src.zip\py4j\java_gateway.py", line 1133, in __call__
  

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:57693)
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2910, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-32-3933876fc135>", line 2, in <module>
    rf = RandomForestClassifier(labelCol="indexedConcatCategory", featuresCol="vector_feature").fit(train).transform(train)
  File "D:\spark\spark-2.2.0-bin-hadoop2.7\python\pyspark\ml\base.py", line 64, in fit
    return self._fit(dataset)
  File "D:\spark\spark-2.2.0-bin-hadoop2.7\python\pyspark\ml\wrapper.py", line 265, in _fit
    java_model = self._fit_java(dataset)
  File "D:\spark\spark-2.2.0-bin-hadoop2.7\python\pyspark\ml\wrapper.py", line 262, in _fit_java
    return self._java_obj.fit(dataset._jdf)
  File "D:\spark\spark-2.2.0-bin-hadoop2.7\python\lib\py4j-0.10.4-src.zip\py4j\java_gateway.py", line 1133, in __call__
  

...

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:57693)
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2910, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-32-3933876fc135>", line 2, in <module>
    rf = RandomForestClassifier(labelCol="indexedConcatCategory", featuresCol="vector_feature").fit(train).transform(train)
  File "D:\spark\spark-2.2.0-bin-hadoop2.7\python\pyspark\ml\base.py", line 64, in fit
    return self._fit(dataset)
  File "D:\spark\spark-2.2.0-bin-hadoop2.7\python\pyspark\ml\wrapper.py", line 265, in _fit
    java_model = self._fit_java(dataset)
  File "D:\spark\spark-2.2.0-bin-hadoop2.7\python\pyspark\ml\wrapper.py", line 262, in _fit_java
    return self._java_obj.fit(dataset._jdf)
  File "D:\spark\spark-2.2.0-bin-hadoop2.7\python\lib\py4j-0.10.4-src.zip\py4j\java_gateway.py", line 1133, in __call__
  

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:57693)
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2910, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-32-3933876fc135>", line 2, in <module>
    rf = RandomForestClassifier(labelCol="indexedConcatCategory", featuresCol="vector_feature").fit(train).transform(train)
  File "D:\spark\spark-2.2.0-bin-hadoop2.7\python\pyspark\ml\base.py", line 64, in fit
    return self._fit(dataset)
  File "D:\spark\spark-2.2.0-bin-hadoop2.7\python\pyspark\ml\wrapper.py", line 265, in _fit
    java_model = self._fit_java(dataset)
  File "D:\spark\spark-2.2.0-bin-hadoop2.7\python\pyspark\ml\wrapper.py", line 262, in _fit_java
    return self._java_obj.fit(dataset._jdf)
  File "D:\spark\spark-2.2.0-bin-hadoop2.7\python\lib\py4j-0.10.4-src.zip\py4j\java_gateway.py", line 1133, in __call__
  

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:57693)
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2910, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-32-3933876fc135>", line 2, in <module>
    rf = RandomForestClassifier(labelCol="indexedConcatCategory", featuresCol="vector_feature").fit(train).transform(train)
  File "D:\spark\spark-2.2.0-bin-hadoop2.7\python\pyspark\ml\base.py", line 64, in fit
    return self._fit(dataset)
  File "D:\spark\spark-2.2.0-bin-hadoop2.7\python\pyspark\ml\wrapper.py", line 265, in _fit
    java_model = self._fit_java(dataset)
  File "D:\spark\spark-2.2.0-bin-hadoop2.7\python\pyspark\ml\wrapper.py", line 262, in _fit_java
    return self._java_obj.fit(dataset._jdf)
  File "D:\spark\spark-2.2.0-bin-hadoop2.7\python\lib\py4j-0.10.4-src.zip\py4j\java_gateway.py", line 1133, in __call__
  

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:57693)
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2910, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-32-3933876fc135>", line 2, in <module>
    rf = RandomForestClassifier(labelCol="indexedConcatCategory", featuresCol="vector_feature").fit(train).transform(train)
  File "D:\spark\spark-2.2.0-bin-hadoop2.7\python\pyspark\ml\base.py", line 64, in fit
    return self._fit(dataset)
  File "D:\spark\spark-2.2.0-bin-hadoop2.7\python\pyspark\ml\wrapper.py", line 265, in _fit
    java_model = self._fit_java(dataset)
  File "D:\spark\spark-2.2.0-bin-hadoop2.7\python\pyspark\ml\wrapper.py", line 262, in _fit_java
    return self._java_obj.fit(dataset._jdf)
  File "D:\spark\spark-2.2.0-bin-hadoop2.7\python\lib\py4j-0.10.4-src.zip\py4j\java_gateway.py", line 1133, in __call__
  

Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:57693)

In [ ]:
help(RandomForestClassifier)

In [ ]:
help(RandomForestClassifier)

